# Deriving CPU Integer Performance Across SPEC Benchmarks

We retrieve the SPECint2000, 2006, and 2017 benchmarks for further processing:

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# get the source files

!wget -O cint2006.html https://www.spec.org/cpu2006/results/cint2006.html


In [ ]:
!wget -O cint2017.html https://www.spec.org/cpu2017/results/cint2017.html

In [ ]:
!wget -O cint2000.html https://www.spec.org/cpu2000/results/cint2000.html

## Process the SPECint2006 benchmark

We need to parse and clean the data, and drop the peak rate (we are only interested in the base rate)

In [ ]:
df2006 = pd.read_html("cint2006.html")[0]

df2006 = df2006.replace(to_replace=r'HTML.*Config', value='', regex=True)
df2006.columns = ['TEST SPONSOR', 'SYSTEM NAME', 'AUTOPARALLEL', 'PROCESSOR ENABLED CORES', 'PROCESSOR ENABLED CHIPS',
             'PROCESSOR CORES/CHIP', 'PROCESSOR THREADS/CORE', 'RESULTS BASE 2006', 'RESULTS PEAK 2006']

df2006['TEST SPONSOR'] = df2006['TEST SPONSOR'].str.upper() 
df2006['SYSTEM NAME'] = df2006['SYSTEM NAME'].str.upper() 

df2006 = df2006.drop(df2006.columns[[2, 6, 8]], axis=1)

df2006['RESULTS BASE 2006'] = pd.to_numeric(df2006['RESULTS BASE 2006'], errors='coerce')

df2006.dropna(subset=['RESULTS BASE 2006'], inplace=True)
df2006.to_csv("cint2006.csv", index=False)

df2006

## Process the SPECint2017 Benchmark

Same procedure for the 2017 benchmark

In [ ]:
df2017 = pd.read_html("cint2017.html")[0]

df2017 = df2017.replace(to_replace=r'HTML.*Config', value='', regex=True)

df2017.columns = ['TEST SPONSOR', 'SYSTEM NAME', 'PARALLEL', 'BASETHREADS', 'PROCESSOR ENABLED CORES',
             'PROCESSOR ENABLED CHIPS', 'PROCESSOR THREADS/CORE', 'RESULTS BASE 2017', 'RESULTS PEAK 2017', 'ENERGY BASE', 'ENERGY PEAK']

df2017['TEST SPONSOR'] = df2017['TEST SPONSOR'].str.upper() 
df2017['SYSTEM NAME'] = df2017['SYSTEM NAME'].str.upper() 

df2017['RESULTS BASE 2017'] = pd.to_numeric(df2017['RESULTS BASE 2017'], errors='coerce')

df2017 = df2017.drop(df2017.columns[[2, 3, 8, 9, 10]], axis=1)

df2017.dropna(subset=['RESULTS BASE 2017'], inplace=True)

df2017.to_csv("cint2017.csv", index=False)

df2017

## Process the SPECint2000 Benchmark

Same process for SPECint2000; the cores/chips/threads are not broken out separately so we have to do some parsing

In [ ]:
df2000 = pd.read_html("cint2000.html")[1]

df2000 = df2000.drop(df2000.columns[[5]], axis=1)
df2000.columns = ['TEST SPONSOR', 'SYSTEM NAME', 'CPU', 'RESULTS BASE 2000', 'RESULTS PEAK 2000']

df2000['TEST SPONSOR'] = df2000['TEST SPONSOR'].str.upper() 
df2000['SYSTEM NAME'] = df2000['SYSTEM NAME'].str.upper() 

df2000['RESULTS BASE 2000'] = pd.to_numeric(df2000['RESULTS BASE 2000'], errors='coerce')

df2000[['PROCESSOR ENABLED CORES', 'PROCESSOR ENABLED CHIPS', 'PROCESSOR THREADS/CORE', 'FOO']] = df2000['CPU'].str.split(', ', n=3, expand=True)

df2000 = df2000.drop(df2000.columns[[2, 4, 8]], axis=1)

df2000 = df2000.replace({'PROCESSOR ENABLED CORES': r'\D'}, {'PROCESSOR ENABLED CORES': ''}, regex=True)
df2000 = df2000.replace({'PROCESSOR ENABLED CHIPS': r'\D'}, {'PROCESSOR ENABLED CHIPS': ''}, regex=True)
df2000 = df2000.replace({'PROCESSOR THREADS/CORE': r'\D'}, {'PROCESSOR THREADS/CORE': ''}, regex=True)

df2000.dropna(subset=['RESULTS BASE 2000'], inplace=True)

df2000.to_csv("cint2000.csv", index=False)

df2000

## Determine the conversion factor of cint2006 to cint2017

There is no direct derivation of cint2017 from cint2006, but by taking the systems which are reported in both cint2006 and cint2017, we can estimate a conversion factor. The ratio between cint2006 and cint2017 can easily be calculated for systems which are present in both benchmarks.

In [ ]:
# join the 2006 and 2017 columns by system name (inner join)
dfm = pd.merge(df2006, df2017, on=['SYSTEM NAME', 'TEST SPONSOR', 'PROCESSOR ENABLED CORES', 'PROCESSOR ENABLED CHIPS'], how='inner')

dfm = dfm.apply(pd.to_numeric, errors='ignore')

dfm

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

d2 = dfm[['RESULTS BASE 2017', 'RESULTS BASE 2006', ]].copy()

X = d2.iloc[:, 0].values.reshape(-1, 1)
Y = d2.iloc[:, 1].values.reshape(-1, 1)
linear_regressor = LinearRegression()
linear_regressor.fit(X, Y)
Y_pred = linear_regressor.predict(X)

plt.scatter(X, Y)
plt.plot(X, Y_pred, color='red')
plt.show()

print('Coefficient/Intercept: %f %f\n' % ( linear_regressor.coef_, linear_regressor.intercept_) )

We also need the reverse, so we can convert SPECint2006 to SPECint2017 without getting negative values.

In [ ]:
d2 = dfm[['RESULTS BASE 2006', 'RESULTS BASE 2017', ]].copy()

X = d2.iloc[:, 0].values.reshape(-1, 1)
Y = d2.iloc[:, 1].values.reshape(-1, 1)
linear_regressor = LinearRegression()
linear_regressor.fit(X, Y)
Y_pred = linear_regressor.predict(X)

plt.scatter(X, Y)
plt.plot(X, Y_pred, color='red')
plt.show()

print('Coefficient/Intercept: %f %f\n' % ( linear_regressor.coef_, linear_regressor.intercept_) )

We can see based on the best-fit line that the conversion factor from cint2017 to cint2006 is 8.22x + 2.39.

To do the reverse (cint2006 to cint2017) we can't simply reverse the above coefficients because it results in negative cint2017 values, so we have to also do a linear regression on the reverse, and we get 0.105x + 0.833.

## Merging the cint2006 and cint2017 Results

In [ ]:
# merge the 2006 and 2017 results

dfm3 = pd.merge(df2006, df2017, on=['SYSTEM NAME', 'TEST SPONSOR', 'PROCESSOR ENABLED CORES', 'PROCESSOR ENABLED CHIPS'], how='outer')

dfm3 = dfm3.apply(pd.to_numeric, errors='ignore')

# if RESULTS BASE 2017 is NaN, replace with derived results from RESULTS BASE 2006
dfm3['RESULTS BASE 2017'].fillna(dfm3['RESULTS BASE 2006'] * 0.105 + 0.832, inplace=True)
dfm3['RESULTS BASE 2006'].fillna(dfm3['RESULTS BASE 2017'] * 8.22 + 2.39, inplace=True)

dfm3['PROCESSOR CORES/CHIP'].fillna(dfm3['PROCESSOR ENABLED CORES'] / dfm3['PROCESSOR ENABLED CHIPS'], inplace=True)
dfm3['PROCESSOR THREADS/CORE'].fillna(1, inplace=True)

dfm3

## Now Merge in the cint2000 Results

For these, we don't have to derive a conversion as the website https://mrob.com/pub/comp/benchmarks/spec.html publishes a conversion factor.

In [ ]:
dfm4 = pd.merge(dfm3, df2000, on=['SYSTEM NAME', 'TEST SPONSOR'], how='outer')

# get the cint2006 scores (from https://mrob.com/pub/comp/benchmarks/spec.html)

# if RESULTS BASE 2017 is NaN, replace with derived results from RESULTS BASE 2006
dfm4['RESULTS BASE 2006'].fillna(dfm4['RESULTS BASE 2000'] / 116, inplace=True)
dfm4['RESULTS BASE 2017'].fillna(dfm4['RESULTS BASE 2006'] * 0.105 + 0.832, inplace=True)

# if RESULTS BASE 2000 is NaN, calculate it
dfm4['RESULTS BASE 2000'].fillna(dfm4['RESULTS BASE 2006'] * 116, inplace=True)

dfm4['PROCESSOR ENABLED CORES_x'].fillna(dfm4['PROCESSOR ENABLED CORES_y'], inplace=True)
dfm4['PROCESSOR ENABLED CHIPS_x'].fillna(dfm4['PROCESSOR ENABLED CHIPS_y'], inplace=True)
dfm4['PROCESSOR THREADS/CORE_x'].fillna(dfm4['PROCESSOR THREADS/CORE_y'], inplace=True)

dfm4['PROCESSOR ENABLED CORES_y'].fillna(dfm4['PROCESSOR ENABLED CORES_x'], inplace=True)
dfm4['PROCESSOR ENABLED CHIPS_y'].fillna(dfm4['PROCESSOR ENABLED CHIPS_x'], inplace=True)

dfm4['PROCESSOR CORES/CHIP'].fillna(1, inplace=True)

dfm4 = dfm4.drop(dfm4.columns[[9, 10, 11]], axis=1)

dfm4.columns = ['TEST SPONSOR', 'SYSTEM NAME', 'PROCESSOR ENABLED CORES',
             'PROCESSOR ENABLED CHIPS', 'PROCESSOR CORES/CHIP', 'RESULTS BASE 2006', 'PROCESSOR THREADS/CORE',
                'RESULTS BASE 2017', 'RESULTS BASE 2000' ]

# add a SPECint95 column for really ancient machines
# https://mrob.com/pub/comp/benchmarks/spec.html#CPU_95
dfm4['RESULTS BASE 95'] = dfm4['RESULTS BASE 2000'] * 8.26

dfm4.to_csv("spec.csv", index=False)
dfm4.sort_values(['RESULTS BASE 2017'], ascending=[False])

In [ ]:
pd.set_option('display.max_rows', 200)

dfm4[ (dfm4['TEST SPONSOR'].str.contains('SGI'))].sort_values(['RESULTS BASE 2006'], ascending=[False])

## Deriving a Figure of Merit for Performance
Due to the preponderance of cint2006 data, this will be the basis of the figure of merit. While we are throwing away some data by disregarding cint2017, we have a sufficiently-robust derivation of cint2006 from cint2017.

The derivation of cint2006 from cint2000 is less robust, but as these are much older machines with much lower scores, they are less relevant. Similarly, the conversion from cint2000 to cint95 is similarly less robust, but again these would be (very) old machines.

In [ ]:
dfm4['RESULTS BASE 2006'].describe()

In [ ]:
dfm4.groupby(pd.cut(dfm4['RESULTS BASE 2006'], 10))['RESULTS BASE 2006'].count()